In [1]:
%pip install --upgrade pip
%pip install boto3 prettytable

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import subprocess
import json
import os
import time
from prettytable import PrettyTable

## Step 1: Create session and client

In [3]:
def set_aws_credentials(profile, region_name='us-east-1'):
    result = subprocess.run(f"aws-vault exec {profile} --json", shell=True, capture_output=True)
    credentials = json.loads(result.stdout)

    # Create a session with the retrieved credentials
    session = boto3.session.Session(
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name=region_name        
    )

    return session



## Get Service Scaling Parameters and Status

In [4]:
def getScalingDetails(profile, region_name, cluster_name, service_name):
    # Use the function with your profile to get a session
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS client using the session
    ecs_client = aws_session.client('ecs')
    app_scaling_client = aws_session.client('application-autoscaling')

    # Fetch the details of the specified service
    response = ecs_client.describe_services(
        cluster=cluster_name,
        services=[service_name]
    )

    service_info = response['services'][0]
    desired_tasks = service_info['desiredCount']
    
    # Resource ID format for ECS services: service/<clusterName>/<serviceName>
    resource_id = f"service/{cluster_name}/{service_name}"

    # Check for scalable targets
    scalable_targets = app_scaling_client.describe_scalable_targets(
        ServiceNamespace='ecs',
        ResourceIds=[resource_id],
        ScalableDimension='ecs:service:DesiredCount'
    )
    
    if scalable_targets['ScalableTargets']:
        # Extract min and max tasks from the first scaling policy
        target = scalable_targets['ScalableTargets'][0]
        min_tasks = target['MinCapacity']
        max_tasks = target['MaxCapacity']
        return min_tasks, max_tasks, desired_tasks
    else:
        # No scaling policy, return the current desired count
        return desired_tasks, desired_tasks, desired_tasks


In [5]:
def getLoadBalancingAlgorithm(profile, region_name, cluster_name, service_name):
    # Use the function with your profile to get a session
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS and ELBv2 clients using the session
    ecs_client = aws_session.client('ecs')
    elbv2_client = aws_session.client('elbv2')

    # Fetch the details of the specified service
    response = ecs_client.describe_services(
        cluster=cluster_name,
        services=[service_name]
    )

    service_info = response['services'][0]
    tg_arns = [lb['targetGroupArn'] for lb in service_info['loadBalancers']]
    target_groups = elbv2_client.describe_target_groups(TargetGroupArns=tg_arns)['TargetGroups']
    for target_group in target_groups:
        if target_group['TargetGroupName'] == 'projects-main-fg':
                # Fetch attributes of the target group
                attributes = elbv2_client.describe_target_group_attributes(
                    TargetGroupArn=target_group['TargetGroupArn']
                )
                # Find and return the load balancing algorithm attribute
                for attr in attributes['Attributes']:
                    if attr['Key'] == 'load_balancing.algorithm.type':
                        return attr['Value']
    
    return "Not Found"

In [6]:
# Specify your cluster and service name
cluster_name = 'EcsCluster1-main'
service_name = 'projects-application-fargate-main'

environments = [
    ('acl-playground', 'us-west-2'),  # playground
    ('acl-staging', 'us-west-2'),  # staging
    ('acl-staging', 'us-east-1'),  # preprod?
    ('acl-production', 'us-east-1'),
    ('acl-production', 'eu-central-1'),
    ('acl-production', 'af-south-1'),
    ('acl-production', 'ca-central-1'),
    ('acl-production', 'ap-northeast-1'),
    ('acl-production', 'ap-southeast-1'),
    ('acl-production', 'ap-southeast-2'),
    ('acl-production', 'sa-east-1'),
]

table = PrettyTable()
table.field_names = ["Profile", "Region", "Min", "Max", "Actual", "%", "Algorithm"]

for profile, region in environments:
    min_tasks, max_tasks, desired_tasks = getScalingDetails(profile, region, cluster_name, service_name)
    algorithm = getLoadBalancingAlgorithm(profile, region, cluster_name, service_name)
    percentage = round((desired_tasks-min_tasks)/(max_tasks-min_tasks)*100.,2) if max_tasks > min_tasks else 100
    print(f"{profile} {region} - Min Tasks: {min_tasks}, Max Tasks: {max_tasks}, Actual Tasks: {desired_tasks} ({percentage}%), Algorithm: {algorithm}")
    table.add_row([profile, region, min_tasks, max_tasks, desired_tasks, percentage, algorithm])

print(table)

acl-playground us-west-2 - Min Tasks: 1, Max Tasks: 4, Actual Tasks: 1 (0.0%), Algorithm: least_outstanding_requests
acl-staging us-west-2 - Min Tasks: 1, Max Tasks: 4, Actual Tasks: 3 (66.67%), Algorithm: least_outstanding_requests
acl-staging us-east-1 - Min Tasks: 3, Max Tasks: 3, Actual Tasks: 3 (100%), Algorithm: least_outstanding_requests
acl-production us-east-1 - Min Tasks: 8, Max Tasks: 64, Actual Tasks: 32 (42.86%), Algorithm: least_outstanding_requests
acl-production eu-central-1 - Min Tasks: 8, Max Tasks: 64, Actual Tasks: 61 (94.64%), Algorithm: least_outstanding_requests
acl-production af-south-1 - Min Tasks: 4, Max Tasks: 4, Actual Tasks: 4 (100%), Algorithm: least_outstanding_requests
acl-production ca-central-1 - Min Tasks: 4, Max Tasks: 4, Actual Tasks: 4 (100%), Algorithm: least_outstanding_requests
acl-production ap-northeast-1 - Min Tasks: 4, Max Tasks: 4, Actual Tasks: 4 (100%), Algorithm: least_outstanding_requests
acl-production ap-southeast-1 - Min Tasks: 4, Ma